In [2]:
import bybit
import math
from datetime import datetime
import time
import os
from IPython.display import clear_output

In [3]:
#Response Structures
class BybitResponse:
    def __init__(self, trader, response) -> None:
        self.response = response
        self._getValues()
        self._isValid()
        self._isSuccessful()

    def _isValid(self):
        if len(self.response) != 2:
            raise Exception("Invalid Response")
    
    def _isSuccessful(self):
        if self.ret_code == 0 and self.ret_msg == 'OK':
            print('Response is valid')
            return
        else:
            raise Exception("Invalid Response")

    def _getValues(self):
        self.data = self.response[0]
        self.result = self.data['result']
        self.ret_code = self.data['ret_code']
        self.ret_msg = self.data['ret_msg']

class MyPosition(BybitResponse):
    def __init__(self, trader) -> None:
        response = trader.client.LinearPositions.LinearPositions_myPosition(symbol=trader.symbol).result()
        super().__init__(trader, response)

class Candles(BybitResponse):
    def __init__(self, trader, interval, limit, time) -> None:
        response = trader.client.LinearKline.LinearKline_get(symbol=trader.symbol, interval=interval, limit=limit, **{'from':time}).result()
        super().__init__(trader, response)

class TradingRecord(BybitResponse):
    def __init__(self, trader) -> None:
        response = trader.client.LinearMarket.LinearMarket_trading(symbol=trader.symbol).result()
        super().__init__(trader, response)

class MyOrders(BybitResponse):
    def __init__(self, trader, status) -> None:
        response = trader.client.LinearOrder.LinearOrder_getOrders(symbol=trader.symbol, order_status=status).result()
        super().__init__(trader, response)
        self.orders = self.result['data']

    def getNewOrders(self) -> list:
        orders = self.result['data']
        new_orders = []
        for order in orders:
            if order['order_status'] == 'New':
                new_orders.append(order)
        return new_orders

In [4]:
class DerivativesTrader:
    #Lists of vaild symbols
    #Only support BTCUSDT and ETHUSDT
    symbol_list = ('BTCUSDT', 'ETHUSDT')
    open_threshold = 0.15
    close_threshold = 0.015
    weight = 1.5

    def __init__(self, api_key, api_secret, symbol) -> None:
        if symbol not in DerivativesTrader.symbol_list:
            raise Exception("Invaild Symbol")
        
        if symbol == 'BTCUSDT':
            self.min_interval = 0.5
            self.qty = 0.01
        elif symbol == 'ETHUSDT':
            self.min_interval = 0.05
            self.qty = 0.001
        
        self.client = DerivativesTrader.__makeClient(api_key, api_secret)
        self.symbol = symbol

        self.openTH = DerivativesTrader.open_threshold
        self.closeTH = DerivativesTrader.close_threshold

        #Check if client is valid
        self.getMyPosition()
        
    def __makeClient(key, secret):
        return bybit.bybit(test=False, api_key=key, api_secret=secret)

    def getMyPosition(self) -> MyPosition:
        return MyPosition(self)

    def getMyOrders(self, status='New'):
        orders = MyOrders(trader, status)
        return orders

    def __getServerTime(self) -> float:
        return float(self.client.Common.Common_getTime().result()[0]['time_now'])

    def __getCandles(self, interval='1', limit=99, time='now') -> list:
        if time == 'now':
            time = self.__getServerTime()
        #Load limit or (limit + 1) candels
        #And return limit candles only
        start_time = time - int(interval) * (limit + 1) * 60
        candles = Candles(self, interval=interval, limit=limit + 1, time=start_time)
        
        if len(candles.result) == limit + 1:
            return candles.result[:limit]
        elif len(candles.result) == limit:
            return candles.result
        else:
            raise Exception("Error in the number of candles")

    def __getLastTradingRecord(self) -> tuple:
        record = TradingRecord(self)
        last = record.result[0]

        if last['side'] == 'Buy':
            low = last['price'] - self.min_interval
            high = last['price']
        elif last['side'] == 'Sell':
            low = last['price']
            high = last['price'] + self.min_interval
        else:
            raise Exception('Error on trading record')

        return low, high

    def __getAvgPrice(self, interval='1', limit=99, time='now') -> float:
        total = 0.0
        candles = self.__getCandles(interval=interval, limit=limit, time=time)

        #Maybe unnecessary check?
        if len(candles) != limit:
            raise Exception('Error in the number of candles')

        for candle in candles:
            close_price = candle['close']
            total += close_price

        return total / limit

    def getPPO(self, interval='1', limit=99, time='now') -> float:
        average = self.__getAvgPrice(interval, limit, time)
        low, high = self.__getLastTradingRecord()
        middle = (low + high)/2

        #should buy when the PPO is positive.
        PPO = average - middle
        ratio = PPO / middle * 100
        return PPO, ratio, low, high

    def updateThreshold(self):
        self.openTH, self.closeTH = self.thresholdFunction()
    
    def thresholdFunction(self, x) -> tuple:
        # y = weight^x
        openTH = DerivativesTrader.open_threshold * pow(DerivativesTrader.weight, x)
        closeTH = DerivativesTrader.close_threshold * pow(DerivativesTrader.weight, x)
        return openTH, closeTH

    def checkPositions(self) -> dict:
        positions = self.getMyPosition()
        buySide, sellSide = positions[0], positions[1]

        if buySide['side'] != 'Buy' or sellSide['side'] != 'Sell':
            raise Exception('Error on Position')

        if buySide['size'] != 0 and sellSide['size'] != 0:
            raise Exception('Have both side of positions')
        
        if buySide['size'] == 0 and sellSide['size'] == 0:
            return 'Empty'
        
        if buySide['size'] != 0:
            return buySide
        elif sellSide['size'] != 0:
            return sellSide

    def placeOrder(self, side, qty, price):
        result = self.client.LinearOrder.LinearOrder_new(side=side,symbol=self.symbol,order_type="Limit",qty=qty,price=price,time_in_force="GoodTillCancel",reduce_only=False, close_on_trigger=False).result()

    def cancleAllOrders(self):
        response = BybitResponse(self, self.client.LinearOrder.LinearOrder_cancelAll(symbol=self.symbol).result())

    def replaceOrder(self):
        pass
    
    def start(self):
        self.placeOrder('Buy', 0.001, )
        self.placeOrder('Sell', 0.001, )

        self.nextStep()

    def nextStep(self):
        # POSITION: You must have only one side
        # ORDER: You must have 2 active OPEN orders always

        # Check if you have both sides of positions
        # if so, raise Exception
        position = self.checkPositions()

        #체결되었는지 확인
        orders = self.getMyOrders().orders

        #self.placeOpenOrders()
        #self.placeCloseOrders()

        if position == 'Empty':
            self.openPositions()

        elif position['side'] == 'Buy':
            self.closePositions()
            self.openPositions()

        elif position['side'] == 'Sell':
            pass


    



In [5]:
trader = DerivativesTrader(api_key="MdojZrGYvNXsXf8sxp", api_secret="jLNEDFu2CGJip7h8UlheZVDX1yMI20V8L6LW", symbol='BTCUSDT')

Response is valid


In [8]:
print(trader.getMyOrders().orders)

Response is valid
None
